In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#设置随机种子用于比较两个计算结果
tf.set_random_seed(1)

# 加载数据
mnist = input_data.read_data_sets("E:\sxl_Programs\Python\MNIST_data\MNIST_data", one_hot=True)

# 超参数
lr = 0.0001  # 学习率
training_iters = 5000  # 训练轮数
batch_size = 200

n_inputs = 28  # MNIST data input (img shape: 28*28)
n_steps = 28  # time steps
n_hidden_units = 160  # neurons in hidden layer
n_classes = 10  # MNIST classes (0-9 digits)

# 输入输出
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# 权重
weights = {
    # (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # (10, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

def RNN(X, weights, biases):
    # hidden layer for input to cell
    ########################################
    # transpose the inputs shape from
    # X ==> (128 batch * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])

    # into hidden
    # X_in = (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])

    # cell
    ##########################################
    # basic LSTM Cell.
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    else:
        cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
    # lstm cell is divided into two parts (c_state, h_state)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)

    # You have 2 options for following step.
    # 1: tf.nn.rnn(cell, inputs);
    # 2: tf.nn.dynamic_rnn(cell, inputs).
    # If use option 1, you have to modified the shape of X_in, go and check out this:
    # https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py
    # In here, we go for option 2.
    # dynamic_rnn receive Tensor (batch, steps, inputs) or (steps, batch, inputs) as X_in.
    # Make sure the time_major is changed accordingly.
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)

    # hidden layer for output as the final results
    #############################################
    # results = tf.matmul(final_state[1], weights['out']) + biases['out']

    # # or
    # unpack to list [(batch, outputs)..] * steps
    outputs = tf.unstack(tf.transpose(outputs, [1, 0, 2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']  # shape = (128, 10)\

    return results


pred = RNN(x, weights, biases)  #预测
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) #损失函数
train_op = tf.train.AdamOptimizer(lr).minimize(cost) #优化器

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) #正确率
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) #正确率均值

with tf.Session() as sess:
    # tf.initialize_all_variables() no long valid from
    init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={x: batch_xs, y: batch_ys})
        step += 1
        if step % 20 == 0:
            train_acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys})
            #             print(" Iter%d,Train accuracy: %.3f" % (step,train_acc))
            test_totalAccuracy = 0.0
            irange = int(mnist.test.labels.shape[0] / batch_size)   #数据集共有多少个batch_size
            for i in range(irange):
                test_batch_xs, test_batch_ys = mnist.test.next_batch(batch_size)
                test_batch_xs = test_batch_xs.reshape([batch_size, n_steps, n_inputs])
                feeds = {x: test_batch_xs, y: test_batch_ys}
                test_acc = sess.run(accuracy, feed_dict=feeds)
                test_totalAccuracy += test_acc
            test_totalAccuracy = test_totalAccuracy / (irange)
            print("  Iter%d,Train accuracy: %.3f,Test accuracy: %.3f" % (step, train_acc, test_totalAccuracy))



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:\sxl_Programs\Python\MNIST_data\MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:\sxl_Programs\Python\MNIST_data\MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting E:\sxl_Programs\Python\MNIST_data\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting E:\sxl_Programs\Python\MNIST_data\MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross

KeyboardInterrupt: 

In [3]:
import tensorflow as tf
tf.__version__

'1.10.0'